In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()


In [ ]:
pop_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860N'

pop_query = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årL",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100",
          "101",
          "102",
          "103",
          "104",
          "105",
          "106",
          "107",
          "108",
          "109",
          "110+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(pop_url,json=pop_query)
r.status_code

In [ ]:
json = r.json()['data']
json[0]

In [ ]:
df = pd.DataFrame.from_dict(json)
df[['age','gender','year']] = df['key'].to_list()
df['age'] = df['age'].str.replace('+','')
df['age'] = df['age'].astype(int)
df['pop'] = (df['values'].apply(lambda x : x[0])).astype(int)
df.drop(['key','values'],axis=1,inplace=True)
df


In [ ]:
df['bin_10'] = pd.cut(df['age'],range(-10,120,10),labels=range(0,120,10))
df

In [ ]:
population_df = df.groupby(['year','bin_10','gender'])['pop'].sum().unstack()
population_df.columns=['male','female']
population_df

In [ ]:
total_population = population_df.groupby('year').sum().sum(axis=1)
total_population

In [ ]:
ax = total_population.plot(style='o--',figsize=(18,12),label='total population')
ax.legend()

In [ ]:
male_population = population_df.unstack()['male']
female_population = population_df.unstack()['female']

male_population.loc['1970':]

In [ ]:
title = 'SWEDEN population by gender, 1860 - 2020\nDataSource : scb.se'
ax = female_population.sum(axis=1).plot(figsize=(18,12),style='ro--',title=title)
male_population.sum(axis=1).plot(ax=ax,style='bo--')
plt.legend(['Female','Male'])
plt.ylabel('population size')
plt.savefig('scb_gender_population_by_gender.jpg',format='jpg')

In [ ]:
diff = male_population - female_population
diff

In [ ]:
prop_diff = diff / (male_population + female_population)
prop_diff

In [ ]:
prop = male_population / (male_population + female_population)
prop

In [ ]:
title = 'SWEDEN male surplus of population 1860 - 2020\nDataSource : scb.se'
diff.sum(axis=1).plot(figsize=(18,12),style='o--',title=title)
plt.axhline(0,ls='dashed',color='k')
plt.ylabel('male surplus')
plt.savefig('scb_gender_male_surplus.jpg',format='jpg')

In [ ]:
prop = prop.loc['1970':]
fig,ax = plt.subplots(3,figsize=(18,12),sharex=True)

ax[0].plot(prop[0],label='0-9 years',ls = 'dashdot')
ax[0].plot(prop[10],label='10-19 years',ls = 'dashdot')
ax[0].plot(prop[20],label='20-29 years',ls = 'dashdot')
ax[0].plot(prop[30],label='30-39 years', ls = 'dashdot')
ax[0].legend()

ax[1].plot(prop[40],label='40-49 years',ls = 'dashdot')
ax[1].plot(prop[50],label='50-59 years',ls = 'dashdot')
ax[1].plot(prop[60],label='60-69 years',ls = 'dashdot')
ax[1].plot(prop[70],label='70-79 years', ls = 'dashdot')
ax[1].legend()


ax[2].plot(prop[80],label='80-89 years',ls = 'dashdot')
ax[2].plot(prop[90],label='90-99 years',ls = 'dashdot')
ax[2].plot(prop[100],label='100-109 years',ls = 'dashdot')
ax[2].plot(prop[110],label='110+ years', ls = 'dashdot')
ax[2].legend()


plt.xticks(rotation=90)

plt.suptitle('SWEDEN proportion males in population,per age group, 2000-2020\nDataSource : scb.se')
plt.savefig('scb_gender_age_grp_male_prop.jpg',format='jpg')